In [8]:
import requests
import pandas as pd

# Define the API key and base URL
API_KEY = "e7bedc1e92f0df2d853f65bed93fdcc2"
BASE_URL = "https://api.the-odds-api.com/v4/sports/"

# Define the sport and region
SPORT = "basketball_nba"  # NBA basketball
REGION = "us"  # US region
MARKET = "h2h"  # Head-to-head (moneyline odds)

def fetch_nba_odds():
    """
    Fetch NBA odds data from The Odds API and return as a pandas DataFrame.
    """
    url = f"{BASE_URL}{SPORT}/odds"
    params = {
        "apiKey": API_KEY,
        "regions": REGION,
        "markets": MARKET,
        "oddsFormat": "american"
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        odds_data = response.json()
        
        # Create a list to store the data for the DataFrame
        rows = []
        
        for game in odds_data:
            for bookmaker in game['bookmakers']:
                for outcome in bookmaker['markets'][0]['outcomes']:
                    row = {
                        'home_team': game['home_team'],
                        'away_team': game['away_team'],
                        'bookmaker': bookmaker['title'],
                        'team': outcome['name'],
                        'odds': outcome['price'],
                        'game_time': game['commence_time']
                    }
                    rows.append(row)
        
        # Create DataFrame from the collected data
        df = pd.DataFrame(rows)
        return df
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

# Run function and display the DataFrame
odds_df = fetch_nba_odds()
if odds_df is not None:
    print(odds_df)

                  home_team               away_team     bookmaker  \
0    Minnesota Timberwolves          Boston Celtics  BetOnline.ag   
1    Minnesota Timberwolves          Boston Celtics  BetOnline.ag   
2    Minnesota Timberwolves          Boston Celtics     LowVig.ag   
3    Minnesota Timberwolves          Boston Celtics     LowVig.ag   
4    Minnesota Timberwolves          Boston Celtics       FanDuel   
..                      ...                     ...           ...   
103      Los Angeles Lakers  Portland Trail Blazers        Bovada   
104      Los Angeles Lakers  Portland Trail Blazers     BetRivers   
105      Los Angeles Lakers  Portland Trail Blazers     BetRivers   
106      Los Angeles Lakers  Portland Trail Blazers         BetUS   
107      Los Angeles Lakers  Portland Trail Blazers         BetUS   

                       team  odds             game_time  
0            Boston Celtics  -195  2025-01-03T00:30:00Z  
1    Minnesota Timberwolves   170  2025-01-03T00:30:00Z

In [20]:
def calculate_payout(odds, bet_amount=100):
    if odds > 0:
        profit = (odds/100) * bet_amount
    else:
        profit = (100/abs(odds)) * bet_amount
    return bet_amount + profit

In [21]:
odds_df['payout_on_100'] = odds_df['odds'].apply(calculate_payout)

#Create a list of unique bookmakers
bookies= odds_df['bookmaker'].unique()
bookies

array(['BetOnline.ag', 'LowVig.ag', 'FanDuel', 'DraftKings', 'BetMGM',
       'MyBookie.ag', 'Bovada', 'BetRivers', 'BetUS'], dtype=object)

In [25]:
#Make dataframes for FanDuel, DraftKings, and BetMGM 
# Create FanDuel-only DataFrame
fanduel_odds_df = odds_df[odds_df['bookmaker'] == 'FanDuel']

# Create DraftKings-only DataFrame
draftkings_odds_df = odds_df[odds_df['bookmaker'] == 'DraftKings']

#Create BetMGM-only DataFrame
betmgm_odds_df = odds_df[odds_df['bookmaker'] == 'BetMGM']

# Reset the index for both DataFrames
fanduel_odds_df = fanduel_odds_df.reset_index(drop=True)
draftkings_odds_df = draftkings_odds_df.reset_index(drop=True)
betmgm_odds_df = betmgm_odds_df.reset_index(drop=True)

In [26]:
betmgm_odds_df

,home_team,away_team,bookmaker,team,odds,game_time,payout_on_100
0,Minnesota Timberwolves,Boston Celtics,BetMGM,Boston Celtics,-200,2025-01-03T00:30:00Z,150.000000
1,Minnesota Timberwolves,Boston Celtics,BetMGM,Minnesota Timberwolves,165,2025-01-03T00:30:00Z,265.000000
2,Miami Heat,Indiana Pacers,BetMGM,Indiana Pacers,110,2025-01-03T00:40:00Z,210.000000
3,Miami Heat,Indiana Pacers,BetMGM,Miami Heat,-135,2025-01-03T00:40:00Z,174.074074
4,Milwaukee Bucks,Brooklyn Nets,BetMGM,Brooklyn Nets,550,2025-01-03T01:10:00Z,650.000000
5,Milwaukee Bucks,Brooklyn Nets,BetMGM,Milwaukee Bucks,-800,2025-01-03T01:10:00Z,112.500000
6,Oklahoma City Thunder,Los Angeles Clippers,BetMGM,Los Angeles Clippers,425,2025-01-03T01:10:00Z,525.000000
7,Oklahoma City Thunder,Los Angeles Clippers,BetMGM,Oklahoma City Thunder,-600,2025-01-03T01:10:00Z,116.666667
8,Golden State Warriors,Philadelphia 76ers,BetMGM,Golden State Warriors,-165,2025-01-03T03:10:00Z,160.606061
9,Golden State Warriors,Philadelphia 76ers,BetMGM,Philadelphia 76ers,135,2025-01-03T03:10:00Z,235.000000
